## Required libraries

- numpy                     1.26.4
- pandas                    2.2.3      
- python                    3.9.20 
- scikit-learn              1.5.1
- scipy                     1.13.1

## Data Pre-processing

### Data from T-brain

In [25]:
import os
import pandas as pd
from datetime import datetime

def read_csv(file_path):
    """
    Reads a CSV file into a pandas DataFrame.
    
    Parameters:
        file_path (str): Path to the CSV file.
        
    Returns:
        pd.DataFrame: The loaded DataFrame.
    """
    try:
        df = pd.read_csv(file_path)
        print(f"Successfully read {file_path}")
        return df
    except Exception as e:
        print(f"Error reading {file_path}: {e}")
        return pd.DataFrame()  # Return empty DataFrame on error

def transform_datetime_string_datetime(s):
    """
    Transforms '2024-01-01 06:31:08' to '20240101063108'.
    
    Parameters:
        s (str): The original datetime string.
        
    Returns:
        str: The transformed datetime string.
    """
    try:
        # Parse the string into a datetime object
        dt = datetime.strptime(s, '%Y-%m-%d %H:%M:%S')
        
        # Format the datetime object into the desired string format
        transformed = dt.strftime('%Y%m%d%H%M%S')
        
        return transformed
    except Exception as e:
        print(f"Error transforming datetime string '{s}': {e}")
        return None  # Return None if parsing fails

def average_each10min(df, datetime_col):
    """
    Averages the DataFrame data in 10-minute intervals.
    
    Parameters:
        df (pd.DataFrame): The input DataFrame with a datetime column.
        datetime_col (str): The name of the datetime column.
        
    Returns:
        pd.DataFrame: The averaged DataFrame.
    """
    try:
        # Convert the datetime column to datetime objects
        df[datetime_col] = pd.to_datetime(df[datetime_col], format='%Y-%m-%d %H:%M:%S.%f')
        
        # Set the datetime column as the index
        df.set_index(datetime_col, inplace=True)
        
        # Resample the data in 10-minute intervals and compute the mean
        df_averaged = df.resample('10T').mean().reset_index()
        
        return df_averaged
    except Exception as e:
        print(f"Error averaging data: {e}")
        return pd.DataFrame()  # Return empty DataFrame on error

def combine_train_files(train_data_path, l_number):
    """
    Combines multiple train CSV files for a given L_number.
    
    Parameters:
        train_data_path (str): Path to the TrainData directory.
        l_number (int): The L_number to process.
        
    Returns:
        pd.DataFrame: The combined DataFrame.
    """
    try:
        file_list = []
        for file_name in os.listdir(train_data_path):
            if f"L{l_number}" in str(file_name):
                file_list.append(file_name)
        print(f"file_list: {file_list}")
        
        if not file_list:
            print(f"No files found for L_number {l_number}")
            return pd.DataFrame()
        
        # Read and concatenate all matching CSV files
        df_list = [read_csv(train_data_path + '/' + file) for file in file_list]
        combined_df = pd.concat(df_list, ignore_index=True)
        print(f"Combined {len(file_list)} files for L_number {l_number}")
        return combined_df
    except Exception as e:
        print(f"Error combining files for L_number {l_number}: {e}")
        return pd.DataFrame()

def process_train_data(train_data_path, processed_path, num_l_numbers=17):
    """
    Processes train data by combining files, transforming datetime, averaging, and saving.
    
    Parameters:
        train_data_path (str): Path to the TrainData directory.
        processed_path (str): Path to save the processed data.
        num_l_numbers (int): Number of L_numbers to process (default is 17).
    """
    try:
        # Create the combined directory if it doesn't exist
        os.makedirs(processed_path, exist_ok=True)
        print(f"processed data will be saved to {processed_path}")
        
        for l_number in range(1, num_l_numbers + 1):
            print(f"\nProcessing L_number: {l_number}")
            if l_number in [2, 4, 7, 8, 9, 10, 12]:
                df = combine_train_files(train_data_path, l_number)
            else:
                path = train_data_path + '/L' + str(l_number) + '_Train.csv'
                df = read_csv(path)
            
            # Perform 10-minute averaging
            averaged_df = average_each10min(
                df,
                datetime_col='DateTime'
            )
            
            print(f"averaged_df done.")
            
            print(averaged_df.columns)
            
            # drop rows with missing value no matter in which column of averaged_df
            averaged_df.dropna(inplace=True)
            
            averaged_df['DateTime_transformed'] = averaged_df.apply(
                lambda row: transform_datetime_string_datetime(str(row['DateTime'])),
                axis=1
            )
            
            print(f"averaged_df['DateTime_transformed'] done.")
            
            print(averaged_df.columns)
            
            # Save the processed DataFrame
            output_file = os.path.join(processed_path, f"L{l_number}_Train_processed.csv")
            averaged_df.to_csv(output_file, index=False)
            print(f"Saved combined and averaged data to {output_file}")
            
    except Exception as e:
        print(f"Error processing train data: {e}")

# ============================ Main Execution =============================

if __name__ == "__main__":
    # Define paths
    TrainData_path = "./TrainData"
    #Combined_path = os.path.join(TrainData_path, "combined")
    Processed_path = os.path.join(TrainData_path, "processed")
    
    # Number of L_numbers to process (e.g., 1 to 17)
    num_l_numbers = 17
    
    # Process the train data
    process_train_data(TrainData_path, Processed_path, num_l_numbers)


processed data will be saved to ./TrainData/processed

Processing L_number: 1
Successfully read ./TrainData/L1_Train.csv
averaged_df done.
Index(['DateTime', 'LocationCode', 'WindSpeed(m/s)', 'Pressure(hpa)',
       'Temperature(°C)', 'Humidity(%)', 'Sunlight(Lux)', 'Power(mW)'],
      dtype='object')
averaged_df['DateTime_transformed'] done.
Index(['DateTime', 'LocationCode', 'WindSpeed(m/s)', 'Pressure(hpa)',
       'Temperature(°C)', 'Humidity(%)', 'Sunlight(Lux)', 'Power(mW)',
       'DateTime_transformed'],
      dtype='object')


/var/folders/tg/l8s3lr394p99bs171s50q_n80000gn/T/ipykernel_57331/2383789346.py:64: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  df_averaged = df.resample('10T').mean().reset_index()


Saved combined and averaged data to ./TrainData/processed/L1_Train_processed.csv

Processing L_number: 2
file_list: ['L2_Train_2.csv', 'L2_Train.csv']
Successfully read ./TrainData/L2_Train_2.csv
Successfully read ./TrainData/L2_Train.csv
Combined 2 files for L_number 2
averaged_df done.
Index(['DateTime', 'LocationCode', 'WindSpeed(m/s)', 'Pressure(hpa)',
       'Temperature(°C)', 'Humidity(%)', 'Sunlight(Lux)', 'Power(mW)'],
      dtype='object')


/var/folders/tg/l8s3lr394p99bs171s50q_n80000gn/T/ipykernel_57331/2383789346.py:64: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  df_averaged = df.resample('10T').mean().reset_index()


averaged_df['DateTime_transformed'] done.
Index(['DateTime', 'LocationCode', 'WindSpeed(m/s)', 'Pressure(hpa)',
       'Temperature(°C)', 'Humidity(%)', 'Sunlight(Lux)', 'Power(mW)',
       'DateTime_transformed'],
      dtype='object')
Saved combined and averaged data to ./TrainData/processed/L2_Train_processed.csv

Processing L_number: 3
Successfully read ./TrainData/L3_Train.csv
averaged_df done.
Index(['DateTime', 'LocationCode', 'WindSpeed(m/s)', 'Pressure(hpa)',
       'Temperature(°C)', 'Humidity(%)', 'Sunlight(Lux)', 'Power(mW)'],
      dtype='object')
averaged_df['DateTime_transformed'] done.
Index(['DateTime', 'LocationCode', 'WindSpeed(m/s)', 'Pressure(hpa)',
       'Temperature(°C)', 'Humidity(%)', 'Sunlight(Lux)', 'Power(mW)',
       'DateTime_transformed'],
      dtype='object')
Saved combined and averaged data to ./TrainData/processed/L3_Train_processed.csv

Processing L_number: 4
file_list: ['L4_Train.csv', 'L4_Train_2.csv']
Successfully read ./TrainData/L4_Train.csv
Su

...

averaged_df done.
Index(['DateTime', 'LocationCode', 'WindSpeed(m/s)', 'Pressure(hpa)',
       'Temperature(°C)', 'Humidity(%)', 'Sunlight(Lux)', 'Power(mW)'],
      dtype='object')
averaged_df['DateTime_transformed'] done.
Index(['DateTime', 'LocationCode', 'WindSpeed(m/s)', 'Pressure(hpa)',
       'Temperature(°C)', 'Humidity(%)', 'Sunlight(Lux)', 'Power(mW)',
       'DateTime_transformed'],
      dtype='object')
Saved combined and averaged data to ./TrainData/processed/L13_Train_processed.csv

Processing L_number: 14
Successfully read ./TrainData/L14_Train.csv
averaged_df done.
Index(['DateTime', 'LocationCode', 'WindSpeed(m/s)', 'Pressure(hpa)',
       'Temperature(°C)', 'Humidity(%)', 'Sunlight(Lux)', 'Power(mW)'],
      dtype='object')


/var/folders/tg/l8s3lr394p99bs171s50q_n80000gn/T/ipykernel_57331/2383789346.py:64: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  df_averaged = df.resample('10T').mean().reset_index()
/var/folders/tg/l8s3lr394p99bs171s50q_n80000gn/T/ipykernel_57331/2383789346.py:64: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  df_averaged = df.resample('10T').mean().reset_index()


averaged_df['DateTime_transformed'] done.
Index(['DateTime', 'LocationCode', 'WindSpeed(m/s)', 'Pressure(hpa)',
       'Temperature(°C)', 'Humidity(%)', 'Sunlight(Lux)', 'Power(mW)',
       'DateTime_transformed'],
      dtype='object')
Saved combined and averaged data to ./TrainData/processed/L14_Train_processed.csv

Processing L_number: 15
Successfully read ./TrainData/L15_Train.csv
averaged_df done.
Index(['DateTime', 'LocationCode', 'WindSpeed(m/s)', 'Pressure(hpa)',
       'Temperature(°C)', 'Humidity(%)', 'Sunlight(Lux)', 'Power(mW)'],
      dtype='object')
averaged_df['DateTime_transformed'] done.
Index(['DateTime', 'LocationCode', 'WindSpeed(m/s)', 'Pressure(hpa)',
       'Temperature(°C)', 'Humidity(%)', 'Sunlight(Lux)', 'Power(mW)',
       'DateTime_transformed'],
      dtype='object')
Saved combined and averaged data to ./TrainData/processed/L15_Train_processed.csv

Processing L_number: 16
Successfully read ./TrainData/L16_Train.csv
averaged_df done.
Index(['DateTime', 'Loca

/var/folders/tg/l8s3lr394p99bs171s50q_n80000gn/T/ipykernel_57331/2383789346.py:64: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  df_averaged = df.resample('10T').mean().reset_index()
/var/folders/tg/l8s3lr394p99bs171s50q_n80000gn/T/ipykernel_57331/2383789346.py:64: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  df_averaged = df.resample('10T').mean().reset_index()


Successfully read ./TrainData/L17_Train.csv
averaged_df done.
Index(['DateTime', 'LocationCode', 'WindSpeed(m/s)', 'Pressure(hpa)',
       'Temperature(°C)', 'Humidity(%)', 'Sunlight(Lux)', 'Power(mW)'],
      dtype='object')
averaged_df['DateTime_transformed'] done.
Index(['DateTime', 'LocationCode', 'WindSpeed(m/s)', 'Pressure(hpa)',
       'Temperature(°C)', 'Humidity(%)', 'Sunlight(Lux)', 'Power(mW)',
       'DateTime_transformed'],
      dtype='object')
Saved combined and averaged data to ./TrainData/processed/L17_Train_processed.csv


### Data from HDARES

In [26]:
import os
import pandas as pd

Datapath = os.path.join(os.getcwd(), 'TrainData', 'additional', f'72T250_item_hour_20241127181859.csv')

# Read the CSV file, skipping problematic lines
try:
    df = pd.read_csv(Datapath, on_bad_lines='skip')  # Skip rows with issues
    print(df.head(10))
except Exception as e:
    print(f"Error loading CSV: {e}")


  location                 time  pressure(hPa)  temperature(C)  \
0   72T250  2022-11-26 01:00:00         1010.3            22.8   
1   72T250  2022-11-26 02:00:00         1009.8            22.2   
2   72T250  2022-11-26 03:00:00         1009.2            22.4   
3   72T250  2022-11-26 04:00:00         1009.0            22.4   
4   72T250  2022-11-26 05:00:00         1009.5            22.4   
5   72T250  2022-11-26 06:00:00         1009.9            22.5   
6   72T250  2022-11-26 07:00:00         1011.0            23.1   
7   72T250  2022-11-26 08:00:00         1011.2            23.6   
8   72T250  2022-11-26 09:00:00         1011.7            23.8   
9   72T250  2022-11-26 10:00:00         1011.5            23.2   

   relativehumidity(%)  averagewindspeed(m/s)  maxgustwind(m/s)  \
0                 92.0                    2.1               5.1   
1                 96.0                    1.2               4.3   
2                 95.0                    0.5               2.5   
3    

In [27]:
import pandas as pd
import numpy as np

# Ensure the 'time' column is in datetime format
df['time'] = pd.to_datetime(df['time'], errors='coerce')

# Check for and handle invalid datetime entries
if df['time'].isnull().any():
    print("Warning: Some entries in the 'time' column could not be converted to datetime.")
    print(df[df['time'].isnull()])  # Print rows with invalid 'time'
    df.dropna(subset=['time'], inplace=True)  # Drop rows with invalid datetime values if needed

# Display the DataFrame
df.head()


,location,time,pressure(hPa),temperature(C),relativehumidity(%),averagewindspeed(m/s),maxgustwind(m/s),precipitation(mm),sunshineduration(hr),SolarRadiation(MJ/m2)
0,72T250,2022-11-26 01:00:00,1010.3,22.8,92.0,2.1,5.1,0.0,0.0,0.0
1,72T250,2022-11-26 02:00:00,1009.8,22.2,96.0,1.2,4.3,0.5,0.0,0.0
2,72T250,2022-11-26 03:00:00,1009.2,22.4,95.0,0.5,2.5,0.0,0.0,0.0
3,72T250,2022-11-26 04:00:00,1009.0,22.4,95.0,0.0,1.1,0.0,0.0,0.0
4,72T250,2022-11-26 05:00:00,1009.5,22.4,94.0,1.0,3.0,0.0,0.0,0.0


In [28]:
df.describe()

,time,pressure(hPa),temperature(C),relativehumidity(%),averagewindspeed(m/s),maxgustwind(m/s),precipitation(mm),sunshineduration(hr),SolarRadiation(MJ/m2)
count,17549,17544.000000,17544.000000,17544.000000,17533.000000,17536.000000,17545.000000,17546.000000,17546.000000
mean,2023-11-27 05:15:14.069177600,1009.666456,23.573957,81.231703,1.076302,3.258468,0.256227,0.298552,0.530821
min,2022-11-26 01:00:00,960.300000,8.200000,32.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2023-05-27 20:00:00,1004.900000,20.100000,72.000000,0.300000,1.500000,0.000000,0.000000,0.000000
50%,2023-11-27 01:00:00,1009.400000,24.000000,84.000000,0.700000,2.500000,0.000000,0.000000,0.020000
75%,2024-05-28 12:00:00,1014.600000,27.000000,91.000000,1.600000,4.400000,0.000000,0.900000,0.670000
max,2024-11-27 18:00:00,1029.100000,35.100000,99.000000,13.900000,32.100000,52.500000,1.000000,3.760000
std,NaN,6.732939,4.922215,12.158549,1.206592,2.440919,1.928194,0.435062,0.893912


In [29]:
import pandas as pd

# Convert 'time' column to datetime 
df['time'] = pd.to_datetime(df['time'], format='%Y-%m-%d %H:%M:%S')

# Set 'time' as the index
df.set_index('time', inplace=True)

# Sort the DataFrame by index to ensure time order
df.sort_index(inplace=True)

df

,location,pressure(hPa),temperature(C),relativehumidity(%),averagewindspeed(m/s),maxgustwind(m/s),precipitation(mm),sunshineduration(hr),SolarRadiation(MJ/m2)
time,,,,,,,,,
2022-11-26 01:00:00,72T250,1010.3,22.8,92.0,2.1,5.1,0.0,0.0,0.00
2022-11-26 02:00:00,72T250,1009.8,22.2,96.0,1.2,4.3,0.5,0.0,0.00
2022-11-26 03:00:00,72T250,1009.2,22.4,95.0,0.5,2.5,0.0,0.0,0.00
2022-11-26 04:00:00,72T250,1009.0,22.4,95.0,0.0,1.1,0.0,0.0,0.00
2022-11-26 05:00:00,72T250,1009.5,22.4,94.0,1.0,3.0,0.0,0.0,0.00
...,...,...,...,...,...,...,...,...,...
2024-11-27 14:00:00,72T250,1015.3,18.8,68.0,1.3,3.0,0.0,0.1,0.39
2024-11-27 15:00:00,72T250,1015.3,19.0,66.0,0.5,2.1,0.0,0.1,0.34
2024-11-27 16:00:00,72T250,1016.1,18.7,66.0,1.4,2.4,0.0,0.0,0.10


#### Resample to 10-minute intervals

In [30]:
# Resample to 10-minute intervals
df_resampled = df.resample('10T').asfreq()

# Display the first 15 rows
df_resampled.head(15)


/var/folders/tg/l8s3lr394p99bs171s50q_n80000gn/T/ipykernel_57331/2010709700.py:2: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  df_resampled = df.resample('10T').asfreq()


,location,pressure(hPa),temperature(C),relativehumidity(%),averagewindspeed(m/s),maxgustwind(m/s),precipitation(mm),sunshineduration(hr),SolarRadiation(MJ/m2)
time,,,,,,,,,
2022-11-26 01:00:00,72T250,1010.3,22.8,92.0,2.1,5.1,0.0,0.0,0.0
2022-11-26 01:10:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-11-26 01:20:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-11-26 01:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-11-26 01:40:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-11-26 01:50:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-11-26 02:00:00,72T250,1009.8,22.2,96.0,1.2,4.3,0.5,0.0,0.0
2022-11-26 02:10:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-11-26 02:20:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Time-based interpolation

In [31]:
# Use time-based interpolation
df_imputed = df_resampled.interpolate(method='time')
df_imputed.head(15)

/var/folders/tg/l8s3lr394p99bs171s50q_n80000gn/T/ipykernel_57331/3058970389.py:2: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df_imputed = df_resampled.interpolate(method='time')


,location,pressure(hPa),temperature(C),relativehumidity(%),averagewindspeed(m/s),maxgustwind(m/s),precipitation(mm),sunshineduration(hr),SolarRadiation(MJ/m2)
time,,,,,,,,,
2022-11-26 01:00:00,72T250,1010.300000,22.800000,92.000000,2.100000,5.100000,0.000000,0.0,0.0
2022-11-26 01:10:00,NaN,1010.216667,22.700000,92.666667,1.950000,4.966667,0.083333,0.0,0.0
2022-11-26 01:20:00,NaN,1010.133333,22.600000,93.333333,1.800000,4.833333,0.166667,0.0,0.0
2022-11-26 01:30:00,NaN,1010.050000,22.500000,94.000000,1.650000,4.700000,0.250000,0.0,0.0
2022-11-26 01:40:00,NaN,1009.966667,22.400000,94.666667,1.500000,4.566667,0.333333,0.0,0.0
2022-11-26 01:50:00,NaN,1009.883333,22.300000,95.333333,1.350000,4.433333,0.416667,0.0,0.0
2022-11-26 02:00:00,72T250,1009.800000,22.200000,96.000000,1.200000,4.300000,0.500000,0.0,0.0
2022-11-26 02:10:00,NaN,1009.700000,22.233333,95.833333,1.083333,4.000000,0.416667,0.0,0.0
2022-11-26 02:20:00,NaN,1009.600000,22.266667,95.666667,0.966667,3.700000,0.333333,0.0,0.0


In [32]:
df_imputed['DateTime'] = df_imputed.index
df_imputed['DateTime_transformed'] = df_imputed['DateTime'].dt.strftime('%Y%m%d%H%M')
df_imputed['location'] = 18
df_imputed

,location,pressure(hPa),temperature(C),relativehumidity(%),averagewindspeed(m/s),maxgustwind(m/s),precipitation(mm),sunshineduration(hr),SolarRadiation(MJ/m2),DateTime,DateTime_transformed
time,,,,,,,,,,,
2022-11-26 01:00:00,18,1010.300000,22.800000,92.000000,2.100000,5.100000,0.000000,0.0,0.000,2022-11-26 01:00:00,202211260100
2022-11-26 01:10:00,18,1010.216667,22.700000,92.666667,1.950000,4.966667,0.083333,0.0,0.000,2022-11-26 01:10:00,202211260110
2022-11-26 01:20:00,18,1010.133333,22.600000,93.333333,1.800000,4.833333,0.166667,0.0,0.000,2022-11-26 01:20:00,202211260120
2022-11-26 01:30:00,18,1010.050000,22.500000,94.000000,1.650000,4.700000,0.250000,0.0,0.000,2022-11-26 01:30:00,202211260130
2022-11-26 01:40:00,18,1009.966667,22.400000,94.666667,1.500000,4.566667,0.333333,0.0,0.000,2022-11-26 01:40:00,202211260140
...,...,...,...,...,...,...,...,...,...,...,...
2024-11-27 17:20:00,18,1016.833333,17.733333,71.000000,1.466667,3.500000,0.000000,0.0,0.020,2024-11-27 17:20:00,202411271720
2024-11-27 17:30:00,18,1016.900000,17.700000,71.000000,1.500000,3.600000,0.000000,0.0,0.015,2024-11-27 17:30:00,202411271730
2024-11-27 17:40:00,18,1016.966667,17.666667,71.000000,1.533333,3.700000,0.000000,0.0,0.010,2024-11-27 17:40:00,202411271740


In [33]:
df_imputed.to_csv('./TrainData/additional/HDARES_imputed_df.csv', index=False)

## Algorithm & Modeling

### Step 1

In [ ]:
import os
import pandas as pd

# Load test data
test_data_path = os.path.join(os.getcwd(), 'ExampleTestData', 'upload(no answer).csv')
TestData = pd.read_csv(test_data_path, encoding='utf-8')
EXquestion = TestData[['序號']].values.flatten()

# Loop through EXquestion in step_size
step_size = 1
dic_for_each_ex = {}
for idx in range(0, len(EXquestion), step_size):
    questionCode = EXquestion[idx]
    print(f'Processing questionCode: {questionCode}')
    
    strLocationCode = str(int(questionCode))[-2:]
    if int(strLocationCode) < 10:
        strLocationCode = strLocationCode[-1]
    print(f"strLocationCode: {strLocationCode}")
        
    datecode = str(int(questionCode))[:12]
    print(f"datecode: {datecode}")
    
    datecode_with_suffix = datecode + '00'  # Adding the '00' suffix
    #print(f"datecode_with_suffix: {datecode_with_suffix}")
    
    labels_for_datecode = {}
    num_paired = 0
    for location in range(1, 18):
        print(f"check matching in location{location}")
        Datapath = os.path.join(os.getcwd(), 'TrainData', 'processed', f'L{location}_Train_processed.csv')
        df = pd.read_csv(Datapath, encoding='utf-8')

        # Ensure no trailing/leading spaces and consistent formatting
        df['DateTime_transformed'] = df['DateTime_transformed'].astype(str).str.strip()
        datecode_with_suffix = str(datecode_with_suffix).strip()

        # Check for exact match
        matching_rows = df[df['DateTime_transformed'] == datecode_with_suffix]
        if not matching_rows.empty:
            print(f"matched with location {location}")
            num_paired += 1
            # Extract the label
            label = matching_rows['Power(mW)'].iloc[0]
            labels_for_datecode[location] = label
    
    print(f"num_paired: {num_paired}")
      
    if labels_for_datecode:
        dic_for_each_ex[questionCode] = labels_for_datecode
    else:
        print(f"No matching datecode found for questionCode: {questionCode}")
        dic_for_each_ex[questionCode] = None

    print(f'Completed corrsponding locations for questionCode: {questionCode}, {idx}/{len(EXquestion)}')


In [11]:
import pickle

# Load variables from the local file
with open('saved_data.pkl', 'rb') as f:
    data = pickle.load(f)

# Extract the variables
#PredictPower = data['PredictPower']
dic_for_each_ex = data['dic_for_each_ex']
#questionCode_not_matched = data['questionCode_not_matched']

print("Data loaded successfully!")

Data loaded successfully!


In [18]:
dic_for_each_ex

{20240117090001: {8: 24.886000000000003, 17: 51.354},
 20240117091001: {8: 26.511000000000003, 17: 97.969},
 20240117092001: {8: 31.68, 17: 141.77100000000002},
 20240117093001: {8: 37.409, 17: 49.427},
 20240117094001: {8: 30.529000000000003, 17: 37.536},
 20240117095001: {8: 36.251, 17: 12.919},
 20240117100001: {8: 54.294, 17: 17.508},
 20240117101001: {8: 75.595, 17: 17.889},
 20240117102001: {8: 97.187, 17: 34.789},
 20240117103001: {8: 90.338, 17: 57.903},
 20240117104001: {8: 59.665, 17: 80.231},
 20240117105001: {8: 38.293, 17: 147.249},
 20240117110001: {8: 15.178999999999998, 17: 180.189},
 20240117111001: {8: 7.967, 17: 289.471},
 20240117112001: {8: 3.954, 17: 259.241},
 20240117113001: {8: 4.739, 17: 215.367},
 20240117114001: {8: 5.327, 17: 121.105},
 20240117115001: {8: 8.184000000000001, 17: 99.986},
 20240117120001: {8: 17.176, 17: 91.928},
 20240117121001: {8: 22.367, 17: 134.221},
 20240117122001: {8: 33.925, 17: 151.096},
 20240117123001: {8: 119.018, 17: 133.493},


In [24]:
# Extract keys as lists for each value (which is an inner dictionary) while checking for None values
unique_key_lists = [sorted(list(value.keys())) for value in dic_for_each_ex.values() if isinstance(value, dict)]

# Remove duplicates by converting to a set and back to a list
unique_key_lists = [list(x) for x in set(tuple(keys) for keys in unique_key_lists)]

# Sort the list of lists to make the order consistent
unique_key_lists.sort()

print(unique_key_lists)
print(len(unique_key_lists))


[[1, 2], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 17], [1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13, 14, 16], [1, 2, 3, 4, 5, 6, 7, 16, 17], [1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 15, 17], [1, 2, 3, 4, 5, 6, 8, 9, 10, 12, 13, 15, 17], [1, 2, 3, 4, 5, 8, 9, 10, 11, 12, 15, 16], [1, 2, 3, 4, 5, 8, 10, 12, 15, 16, 17], [1, 2, 3, 4, 5, 10, 15], [1, 2, 3, 4, 5, 12, 16], [1, 2, 3, 4, 9, 10, 11, 12, 15, 16], [1, 2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 16], [1, 2, 3, 5, 8, 9, 10, 12, 15], [1, 2, 3, 8, 9, 11, 12, 14, 15, 17], [1, 2, 3, 9, 10, 11, 12, 14, 15, 16, 17], [1, 2, 3, 9, 10, 11, 12, 15, 16, 17], [1, 2, 4, 5, 6, 7, 9, 11, 12, 15, 16, 17], [1, 2, 4, 5, 6, 7, 10, 12, 15, 16, 17], [1, 2, 4, 5, 6, 7, 10, 12, 16, 17], [1, 2, 4, 5, 6, 7, 12, 16, 17], [1, 2, 4, 5, 6, 8, 10, 12, 13, 14], [1, 2, 4, 6, 7, 8, 9, 10, 11, 12, 14, 16, 17], [1, 2, 4, 6, 7, 8, 9, 10, 11, 12, 16, 17], [1, 2, 4, 6, 7, 8, 9, 11, 12, 13, 14, 15], [1, 2, 4, 7, 8, 9, 10, 11, 12, 14, 16], [1, 2, 4, 8, 9, 12, 15, 16], [1, 2, 

In [22]:
# Find all keys in dic_for_each_ex with None values
none_keys = [key for key, value in dic_for_each_ex.items() if value is None]

print(none_keys)
print(len(none_keys))


[20240812090008, 20240812091008, 20240812092008, 20240812093008, 20240812094008, 20240812095008, 20240812100008, 20240812101008, 20240812102008, 20240812103008, 20240812104008, 20240812105008, 20240812110008, 20240812111008, 20240812112008, 20240812113008, 20240812114008, 20240812115008, 20240812120008, 20240812121008, 20240812122008, 20240812123008, 20240812124008, 20240812125008, 20240812130008, 20240812131008, 20240812132008, 20240812133008, 20240812134008, 20240812135008, 20240812140008, 20240812141008, 20240812142008, 20240812143008, 20240812144008, 20240812145008, 20240812150008, 20240812151008, 20240812152008, 20240812153008, 20240812154008, 20240812155008, 20240812160008, 20240812161008, 20240812162008, 20240812163008, 20240812164008, 20240812165008, 20240828090008, 20240828091008, 20240828092008, 20240828093008, 20240828094008, 20240828095008, 20240828100008, 20240828101008, 20240828102008, 20240828103008, 20240828104008, 20240828105008, 20240828110008, 20240828111008, 2024082

### Step 2

In [ ]:
import os
import pandas as pd
import joblib
from sklearn.ensemble import GradientBoostingRegressor

# Function to train GradientBoostingRegressor
def train_sklearn_gbdt(features_df, labels):
    # Initialize the model with desired parameters
    gbdt_model = GradientBoostingRegressor(
        n_estimators=200,
        learning_rate=0.05,
        min_samples_split=5,
        min_samples_leaf=3,
        random_state=42
    )
    
    # Fit the model
    gbdt_model.fit(features_df, labels)
    
    return gbdt_model

# Main prediction dictionary
new_predicted_power = {}
n = 1

for question, prediction in dic_for_each_ex.items():
    print(f"Processing question: {question}")
    
    # Extract location code from question
    locationcode = int(str(question)[-2:])
    
    if prediction is None:  # Skip None values
        new_predicted_power[question] = None
        print(f"No matched datecode in any other location for question: {question}")
        n += 1
        continue

    # Load the current location's data
    path = os.path.join(os.getcwd(), 'TrainData', 'processed', f'L{locationcode}_Train_processed.csv')
    current_location_df = pd.read_csv(path, encoding='utf-8')

    # Find matched locations from `prediction`
    matched_locations = list(prediction.keys())

    # Find time intersections between the current location and matched locations
    intersection_time = set(current_location_df['DateTime_transformed']).intersection(
        *[pd.read_csv(
            os.path.join(os.getcwd(), 'TrainData', 'processed', f'L{loc}_Train_processed.csv'),
            encoding='utf-8'
        )['DateTime_transformed'] for loc in matched_locations]
    )

    # Extract time-intersected parts from the current location's data
    intersection_label = current_location_df[
        current_location_df['DateTime_transformed'].isin(intersection_time)
    ]['Power(mW)'].values.flatten()

    # Prepare features for the matched locations
    features_df = pd.DataFrame()
    for location in matched_locations:
        # Load the matched location's data
        data_path = os.path.join(os.getcwd(), 'TrainData', 'processed', f'L{location}_Train_processed.csv')
        location_df = pd.read_csv(data_path, encoding='utf-8')
        
        # Add features for the time intersection
        tmp_intersection_features = location_df[
            location_df['DateTime_transformed'].astype(int).isin(intersection_time)
        ][
            ['WindSpeed(m/s)', 'Pressure(hpa)', 'Temperature(°C)', 'Humidity(%)',
             'Sunlight(Lux)', 'Power(mW)', 'LocationCode']
        ]
        
        # Reset index
        tmp_intersection_features.reset_index(drop=True, inplace=True)
        
        # Rename columns to include the location identifier
        tmp_intersection_features.columns = [
            f"{col}_L{location}" for col in tmp_intersection_features.columns
        ]
        
        # Concatenate with features_df
        features_df = pd.concat([features_df, tmp_intersection_features], axis=1)
        print(f'features_df.shape: {features_df.shape}')

    # Add HDARES data
    HDARES_imputed_df = pd.read_csv('./TrainData/additional/HDARES_imputed_df.csv')
    HDARES_imputed_df['DateTime_transformed'] = HDARES_imputed_df['DateTime_transformed'].astype(str) + '00'
    HDARES_imputed_df['DateTime_transformed'] = HDARES_imputed_df['DateTime_transformed'].astype(int)
    
    tmp_intersection_features = HDARES_imputed_df[
        HDARES_imputed_df['DateTime_transformed'].isin(intersection_time)
    ][
        ['pressure(hPa)', 'temperature(C)', 'relativehumidity(%)', 'averagewindspeed(m/s)',
         'maxgustwind(m/s)', 'precipitation(mm)', 'sunshineduration(hr)', 'SolarRadiation(MJ/m2)']
    ]
    
    # Reset index
    tmp_intersection_features.reset_index(drop=True, inplace=True)
    
    # Concatenate HDARES features with features_df
    features_df = pd.concat([features_df, tmp_intersection_features], axis=1)
    print(f'features_df.shape after adding HDARES: {features_df.shape}')

    # Check if features_df is empty
    if features_df.shape[0] < 1:
        new_predicted_power[question] = None
        print(f"No features_df for question: {question}, features_df.shape[0]: {features_df.shape[0]}")
        n += 1
        continue

    # Prepare features for prediction
    current_question_features_df = pd.DataFrame()
    datecode_with_suffix = str(question)[:12] + '00'  # Adding the '00' suffix

    for location in matched_locations:
        # Load the matched location's data
        data_path = os.path.join(os.getcwd(), 'TrainData', 'processed', f'L{location}_Train_processed.csv')
        location_df = pd.read_csv(data_path, encoding='utf-8')
        
        location_df['DateTime_transformed'] = location_df['DateTime_transformed'].astype(str)
        # Add features for the current date
        tmp_matched_location_features = location_df[
            location_df['DateTime_transformed'] == datecode_with_suffix
        ][
            ['WindSpeed(m/s)', 'Pressure(hpa)', 'Temperature(°C)', 'Humidity(%)',
             'Sunlight(Lux)', 'Power(mW)', 'LocationCode']
        ]
        
        # Reset index
        tmp_matched_location_features.reset_index(drop=True, inplace=True)
        
        # Rename columns
        tmp_matched_location_features.columns = [
            f"{col}_L{location}" for col in tmp_matched_location_features.columns
        ]
        current_question_features_df = pd.concat([current_question_features_df, tmp_matched_location_features], axis=1)
        print(f'current_question_features_df.shape: {current_question_features_df.shape}')

    # Add HDARES data
    tmp_matched_location_features = HDARES_imputed_df[
        HDARES_imputed_df['DateTime_transformed'].astype(str) == datecode_with_suffix
    ][
        ['pressure(hPa)', 'temperature(C)', 'relativehumidity(%)', 'averagewindspeed(m/s)',
         'maxgustwind(m/s)', 'precipitation(mm)', 'sunshineduration(hr)', 'SolarRadiation(MJ/m2)']
    ]
    
    # Reset index
    tmp_matched_location_features.reset_index(drop=True, inplace=True)
    
    # Concatenate HDARES features with current_question_features_df
    current_question_features_df = pd.concat([current_question_features_df, tmp_matched_location_features], axis=1)
    print(f'current_question_features_df.shape after adding HDARES: {current_question_features_df.shape}')

    # Ensure feature columns in current_question_features_df match those in features_df
    current_question_features_df = current_question_features_df.reindex(columns=features_df.columns, fill_value=0)

    # Handle missing values
    features_df.fillna(0, inplace=True)
    current_question_features_df.fillna(0, inplace=True)

    # Convert data types to numeric
    features_df = features_df.apply(pd.to_numeric, errors='coerce')
    current_question_features_df = current_question_features_df.apply(pd.to_numeric, errors='coerce')

    # Fill any NaNs introduced after conversion
    features_df.fillna(0, inplace=True)
    current_question_features_df.fillna(0, inplace=True)

    # Verify feature alignment
    print("Features in training data:", list(features_df.columns))
    print("Features in prediction data:", list(current_question_features_df.columns))

    # Model file name
    model_name = f'gbdt_features{"_".join(map(str, matched_locations))}_labelL{locationcode}.pkl'
    model_path = os.path.join(os.getcwd(), 'model', 'sklearn', model_name)

    # Train or load the model
    if os.path.exists(model_path):
        # Load the existing model
        model = joblib.load(model_path)
        print(f"Loaded existing model: {model_name}")
    else:
        # Train a new Gradient Boosting model using sklearn
        print("Training the model...")
        model = train_sklearn_gbdt(features_df, intersection_label)
        
        # Save the trained model
        os.makedirs(os.path.dirname(model_path), exist_ok=True)
        joblib.dump(model, model_path)
        print(f"Model saved: {model_name}")

    # Predict using the model
    pred = model.predict(current_question_features_df)
    new_predicted_power[question] = pred

    print(f"Prediction for question {question}: {pred}")
    n += 1
    print(f"Completed {n}/{len(dic_for_each_ex)}\n")

print("All predictions completed.")
